<a href="https://colab.research.google.com/github/ryans94/CS584/blob/main/Ryan_Smith_hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 584 :: Data Mining :: George Mason University :: Fall 2025


# Homework 2: Linear Regression&Neural Networks

- **100 points [6% of your final grade]**
- **Due Sunday, October 19 by 11:59pm**

- *Goals of this homework:* (1) implement the linear regression model; (2) implement the multi-layer perceptron neural network; (3) tune the hyperparameters of MLP model to produce classification result as good as possible.

- *Submission instructions:* for this homework, you should submit your notebook file to **Canvas** (look for the homework 2 assignment there). Please name your submission **FirstName_Lastname_hw2.ipynb**, so for example, my submission would be something like **Ziwei_Zhu_hw2.ipynb**. Your notebook should be fully executed so that we can see all outputs.

## Part 1: Linear Regression (40 points)

Recent studies have found that novel mobile games can lead to increased physical activity. A notable example is Pokemon Go, a mobile game combining the Pokemon world through augmented reality with the real world requiring players to physically move around. Specifically, in the following study, researchers have found that Pokemon Go leads to increased levels of physical activity for the most engaged players! https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5174727/
![image.png](attachment:image.png)



In this part, our goal is to predict the combat point of each pokemon in the 2017 Pokemon Go mobile game. Each pokemon has its own unique attributes that can help predicting its combat points. These include:

- Stamina
- Attack value
- Defense value
- Capture rate
- Flee rate
- Spawn chance
- Primary strength

The file pokemon_data.csv contains data of 146 pokemons to be used in this homework. The rows of these files refer to the data samples (i.e., pokemon samples), while the columns denote the name of the pokemon (column 1), its attributes (columns 2-8), and the combat point outcome (column 9). You can ignore column 1 for the rest of this problem.

First, let's load the data by executing the following code.

**Note: you need to install the pandas library beforehand**

In [ ]:
import numpy as np
import pandas as pd

data_frame = pd.read_csv('pokemon_data.csv')
data_frame.head()

Pandas is a fast, powerful, flexible and easy-to-use open-source data analysis and manipulation tool, built on top of Python. By executing the following code, let's create one Numpy array to contain the feature data without the name column and one array to contain the combat point ground truth.

In [ ]:
features = data_frame.values[:, 1:-1]
labels = data_frame.values[:, -1]
print('array of labels: shape ' + str(np.shape(labels)))
print('array of feature matrix: shape ' + str(np.shape(features)))

Now, you may find out that we have a categorical feature 'primary_strength' in our data. Categorical features require special attention because usually they cannot be the input of regression models as they are. A potential way to treat categorical features is to simply convert each value of the feature to a separate number. However, this might impute non-existent relative associations between the features, which might not always be representative of the data (e.g., if we assign “1” to the value “green” and “2” to the value “red”, the regression algorithm will assume that “red” is greater than “green,” which is not necessarily the case). For this reason, we can use a “one hot encoding” to represent categorical features. According to this, we will create a binary column for each category of the categorical feature, which will take a value of 1 if the sample belongs to that category, and 0 otherwise. For each categorical feature of the problem, count the number of different values and implement the one hot encoding. For the remaining of the problem, you will be working with the one hot encoding of the categorical features.


In the next cell, write your code to replace the categorical feature 'primary_strength' with **one-hot encoding** and generate the new version of the Numpy array 'features'.

**Hint: if you don't remember one hot encoding, review the slides of our first-week lecture.**

**Note: do not use sklearn to automatically generate one hot encoding.**

In [ ]:
# Write your code here

Besides, you may also notice that other features have different scales. So, you need to standardize them: $({x-\mu})/{\sigma}$, where $\mu$ is the mean and $\sigma$ is the standard deviation. Write your code below.

**Hint: details about feature standardization is also in slides of our first-week lecture.**

**Note: You do not need to do standardize for one-hot encodings.**

In [ ]:
# Write your code here

Now, in the next cell, you need to implement your own **linear regression model** using the **Ordinary Least Squares (OLS)** solution **without regularization**. And here, you should adopt the **5-fold cross-validation** method. For each fold compute and print out the **square root** of the residual sum of squares error (RSS) between the actual and predicted outcome variable. Also compute and print out the **average** square root of the RSS over all folds.

**Note: You should implement the algorithm by yourself. You are NOT allowed to use Machine Learning libraries like Sklearn.**

**Hint: Use numpy.linalg.pinv() for calculating the inverse of a matrix.**

**Hint: details about cross-validation is in slides of KNN lecture.**

**Hint: the sqrt_RSS for each fold should be 0~3000**


In [ ]:
# Write your code here

At the end in this part, please repeat the same experiment as in the previous step, but instead of linear regression, implement linear regression **with L2-norm regularization**. Experiment and report your results (average square root of RSS over 5-fold cross-validation) with different values of the regularization term $\lambda=\{1, 0.1, 0.01, 0.001, 0.0001\}$.

**Hint: details about the closed-form solution with regularization is on page 74 in slides of our linear regression lecture.**

In [ ]:
# Write your code here

## Part 2: Neural Networks (40 points)

In this part, you are going to implement your multi-layer perceptron model by the Pytorch library. You will still use the same handwritten digit image dataset from HW1. So, in the next few cells, please run the provided code to load and process the data, and create dataset objects for further use by Pytorch.

**Note: you need to install Pytorch beforehand (check out https://pytorch.org/get-started/locally/). Or, you can use Google Colab for this homework, which is recommended.**

In [ ]:
# load data from file and split into training and validation sets
import numpy as np
data = np.loadtxt("train.txt", delimiter=',')
perm_idx = np.random.permutation(data.shape[0])
vali_num = int(data.shape[0] * 0.2)
vali_idx = perm_idx[:vali_num]
train_idx = perm_idx[vali_num:]
train_data = data[train_idx]
vali_data = data[vali_idx]
train_features = train_data[:, 1:].astype(np.float32)
train_labels = train_data[:, 0].astype(int)
vali_features = vali_data[:, 1:].astype(np.float32)
vali_labels = vali_data[:, 0].astype(int)

In [ ]:
# define a Dataset class
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
class MNISTDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx, :], self.labels[idx]

In [ ]:
training_data = MNISTDataset(train_features, train_labels)
vali_data = MNISTDataset(vali_features, vali_labels)
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
vali_dataloader = DataLoader(vali_data, batch_size=batch_size)

for X, y in train_dataloader:
    print(f"Shape of X [N, F]: {X.shape} {X.dtype}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Now, you should have the train_dataloader and vali_dataloader. Then, you need to build and train your multi-layer perceptron model by Pytorch.

https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html gives a comprehansive example how to achieve this. Please read this tutorial closely, and implement the model in the next few cells.

https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/c30c1dcf2bc20119bcda7e734ce0eb42/quickstart_tutorial.ipynb provides the interactive version, which you can run and edit.

**Note: in your implementation:**
- you will only have three layers [784 -> 512 -> 10], you need to remove the [512 -> 512] layer in the tutorial.
- add 'weight_decay=1e-4' in torch.optim.SGD to add L2 regularization.
- train the model for 10 epochs instead of 5 epochs.
- keep all other hyper-parameters the same as used in the tutorial.
- **You are allowed to resue the code in the tutorial for this homework**


**Note: print out the training process and the final accuracy on the validation set.**

**Note: you can use Colab for running the code with GPU for free (open a colab notebook, then Runtime->Change runtime type->Hardware accelerator->GPU)**

In [ ]:
# Write your code

## Part 3: Tune Hyperparameter (20 points)

In this part, you need to do your best to tune the hyperparameters in the MLP to build the best model and evaluate the predictions for the testing data.

Now, you should tune four hyperparameters:

- the number of layers and the dimension of each layer (explore as much as you can, but choose reasonable settings considering the computational resources you have)
- the activation function (choose from sigmoid, tanh, relu)
- weight decay
- number of training epochs


**Hint: You can tune these hyperparameters by one randomly generated validation set, or you can also use the cross-validation method.**

**Note: you can use Colab for running the code with GPU for free**


In [ ]:
# Write your code here

### Question: What is your final hyperparameter setting? How do you tune them? What choices have you tried?

#### Write your answer here

Please apply the model with the best hyperparameter setting you find above to the testing set. First of all, let's load the testing data by executing the following code.

In [ ]:
test_features = np.loadtxt("test.txt", delimiter=',')
test_labels = np.loadtxt("test_label.txt")
print('array of testing feature matrix: shape ' + str(np.shape(test_features)))
print('array of testing labels: shape ' + str(np.shape(test_labels)))

Now, report the Accuracy of your best model on the testing set.

**Hint: use the following two lines of code to generate the label predictions for test data:**
- raw_pred = model(torch.tensor(test_features).to(device).float())
- pred = np.argmax(raw_pred.to('cpu').detach().numpy(), axis=1)

In [ ]:
# Write your code here